In [2]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [3]:
import tenpy
import tenpy.linalg.np_conserved as npc
from tenpy.algorithms import dmrg
from tenpy.networks.mps import MPS

In [4]:
from tenpy.networks.site import SpinSite, SpinHalfSite
from tenpy.models.lattice import Chain
from tenpy.models.model import CouplingModel, NearestNeighborModel, MPOModel

In [5]:
class ClusterIsing(CouplingModel):
    def __init__(self, model_params):
        # Read out/set default parameters
        name = "Cluster Ising model"
        L = model_params.get('L', 5)
        B = model_params.get('B', 0)
        bc_MPS = model_params.get('bc_MPS', 'finite')
        
        # sites
        site = SpinHalfSite(conserve=None)

        # lattice
        bc = 'open'
        lat = Chain(L, site, bc=bc, bc_MPS=bc_MPS)

        # initialize CouplingModel
        CouplingModel.__init__(self, lat)

        # add terms of the Hamiltonian
        self.add_onsite(-B, 0, 'Sigmax')
        self.add_multi_coupling(
            -1,
            [
                ('Sigmaz', -1, 0),
                ('Sigmax', 0, 0),
                ('Sigmaz', 1, 0),
            ]
        )

        # initialize H_MPO
        MPOModel.__init__(self, lat, self.calc_H_MPO())

In [6]:
dmrg_params = {
    "trunc_params": {"chi_max": 8, "chi_min": 1, "svd_min": 1.e-10},
    "min_sweeps":100,
    "max_sweeps":200,
    "mixer": True,
    "combine":False,
    'decay':2,
    'amplitude':10e-1,
    'disable_after':60,
    'update_env':0
}

In [7]:
B_params = np.linspace(1.3, 2, 8)

In [8]:
B_params

array([1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. ])

In [9]:
import h5py
from tenpy.tools import hdf5_io

In [10]:
wavefunctions = list()
energies = list()

for B in B_params:
    print("Commencing B={:.2f} model".format(B))
    model=ClusterIsing({'B': B, 'L':200})
    
    psi = MPS.from_desired_bond_dimension(model.lat.mps_sites(), 1, bc=model.lat.bc_MPS)
    psi.canonical_form()

    eng = dmrg.TwoSiteDMRGEngine(psi, model, dmrg_params)
    e, psi = eng.run()

    wavefunctions.append(psi)
    energies.append(e)

    data = {
        "wavefunction": psi,
        "paramters": {"B": B}
    }

    filename = r"data/transverse_cluster_200_site_dmrg/{:.2f}".format(B).replace(".", "_")
    filename += ".h5"
    with h5py.File(filename, 'w') as f:
        hdf5_io.save_to_hdf5(f, data)

Commencing B=1.30 model


final DMRG state not in canonical form up to norm_tol=1.00e-05: norm_err=4.78e-05


Commencing B=1.40 model


/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv/lib/python3.12/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))
final DMRG state not in canonical form up to norm_tol=1.00e-05: norm_err=2.80e-05


Commencing B=1.50 model


...

/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv/lib/python3.12/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


Commencing B=1.90 model


/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv/lib/python3.12/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


Commencing B=2.00 model


/home/kieranc/Desktop/numerical_spt_classification/code/num_spt_venv/lib/python3.12/site-packages/tenpy/tools/params.py:230: UserWarning: unused options for config TwoSiteDMRGEngine:
['amplitude', 'decay', 'disable_after', 'update_env']
  warnings.warn(msg.format(keys=sorted(unused), name=self.name))


In [11]:
7

7